In [1]:
## Necessary libraries
import time
import json
import numpy as np
import pandas as pd
from pandas import json_normalize

import datetime

import seaborn as sns
import matplotlib.pyplot as plt

import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

## DATA LOADING
#### Since in the dataset thier are some json columns so we need to define a function to load them as given below

In [2]:

def load_df(csv_path, nrows = None):
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(csv_path,
                     #converters are dict of functions for converting values in certain columns. Keys can either be integers or column labels.
                     #json.loads() method can be used to parse a valid JSON string and convert it into a Python Dictionary.
                     #It is mainly used for deserializing native string, byte, or byte array which consists of JSON data into Python Dictionary.
                     converters = {col: json.loads for col in json_cols},                                                                         
                         dtype = {'fullVisitorId': 'str'}, # Important!!
                         nrows = nrows)
    for col in json_cols:
        # for each column, flatten data frame such that the values of a single col are spread in different cols
        # This will use subcol as names of flat_col.columns
        flat_col = json_normalize(df[col])
        # Name the columns in this flatten data frame as col.subcol for tracability
        flat_col.columns = [f"{col}.{subcol}" for subcol in flat_col.columns]
        # Drop the json_col and instead add the new flat_col
        df = df.drop(col, axis = 1).merge(flat_col, right_index = True, left_index = True)
    return df



In [3]:
csv_train_path = 'train_v2.csv'
csv_test_path = 'test_v2.csv'

In [4]:
%%time
# %%time is used to calculate the timing of code chunk execution #
train = load_df(csv_train_path, nrows = 120000)
test = load_df(csv_test_path, nrows = None)
train.shape, test.shape

CPU times: total: 1min 12s
Wall time: 1min 37s


((120000, 60), (401589, 59))

#### Since we are implementing this using cpu so we have taken 120000 rows while the real shape is as follows:-
#### Train dataset shape is : (1708337, 60)
#### Test dataset shape is : (401589, 59)
#### Here each record corresponds to one visit to store.

## DESCRIBING DATA

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 60 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    120000 non-null  object
 1   customDimensions                                   120000 non-null  object
 2   date                                               120000 non-null  int64 
 3   fullVisitorId                                      120000 non-null  object
 4   hits                                               120000 non-null  object
 5   socialEngagementType                               120000 non-null  object
 6   visitId                                            120000 non-null  int64 
 7   visitNumber                                        120000 non-null  int64 
 8   visitStartTime                                     120000 non-null  int64 
 9   devi

In [6]:
train.loc[:2]

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Firefox,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,...,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,...,not available in demo dataset,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### We can observe from above that many columns has a huge amount of null values so we have the following options to clean the data :
#### 1. Get rid of the corresponding rows
#### 2. Get rid of the whole column
#### 3. Set the values to some values such as zero,mean,median etc.
## DATA PREPROCESSING
#### NOTE - For further data cleaning we need to get rid of values such as 'unknown.unknown', '(not set)', 'not available in demo dataset', '(not provided)', '(none)', 'NA' in the training dataset so that data cleaning and preproccessing can be done effiently

In [7]:
unknown_values = ['unknown.unknown', '(not set)', 'not available in demo dataset', '(not provided)', '(none)', '<NA>']

In [8]:
train.replace(unknown_values, np.nan, inplace=True)

In [9]:
train.loc[:2]

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508198450,1,1508198450,Firefox,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,"[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,1508176307,6,1508176307,Chrome,...,NaN,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,1508201613,1,1508201613,Chrome,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### It can be observed that all the values which were null basically have been replaced to NAN
#### We also need to drop such columns who has only one unique value ,basically it means if any column has all values same then it is a redundant feature

In [10]:
unique_value_counts = train.nunique()
unique_value_counts

channelGrouping                                           8
customDimensions                                          6
date                                                     46
fullVisitorId                                        107402
hits                                                 110795
socialEngagementType                                      1
visitId                                              117682
visitNumber                                             221
visitStartTime                                       117673
device.browser                                           34
device.browserVersion                                     0
device.browserSize                                        0
device.operatingSystem                                   17
device.operatingSystemVersion                             0
device.isMobile                                           2
device.mobileDeviceBranding                               0
device.mobileDeviceModel                

In [11]:
columns_to_drop = unique_value_counts[unique_value_counts == 1].index
train.drop(columns=columns_to_drop, inplace=True)

In [12]:
train.shape

(120000, 53)

#### It can be oberved that after dropping the columns with one unique values ,the shape regarding columns has been reduced from 60 -> 53

In [14]:
# Converting date column from character to Date class.
train['date'] = pd.to_datetime(train['date'], format='%Y%m%d')

In [15]:
train['date']

0        2017-10-16
1        2017-10-16
2        2017-10-16
3        2017-10-16
4        2017-10-16
            ...    
119995   2016-12-09
119996   2016-12-09
119997   2016-12-09
119998   2016-12-09
119999   2016-12-09
Name: date, Length: 120000, dtype: datetime64[ns]

In [21]:
# Converting all the newly JSON columns (hits, pageviews, transactionRevenue) from character to numeric
columns_to_convert = ['totals.hits', 'totals.pageviews', 'totals.transactionRevenue']
for column in columns_to_convert:
    train[column] = pd.to_numeric(train[column], errors='coerce')
# The 'errors' parameter is set to 'coerce' to replace any non-convertible values with NaN.
# If you want to keep the non-convertible values as they are, you can omit the 'errors' parameter.


In [24]:
# Calculate the percentage of missing values for each column
missing_percentage = (train.isnull().sum() / len(train)) * 100

# Define a threshold for missing values (95% in this case)
threshold = 95

# Get the list of columns to drop based on the threshold
columns_to_drop = missing_percentage[missing_percentage > threshold].index

columns_to_drop

Index(['hits', 'device.browserVersion', 'device.browserSize',
       'device.operatingSystemVersion', 'device.mobileDeviceBranding',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.flashVersion', 'device.language', 'device.screenColors',
       'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation',
       'totals.transactions', 'totals.transactionRevenue',
       'totals.totalTransactionRevenue',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adContent'],
      dtype='object')

In [26]:
# Drop the columns with more than 95% missing values
train.drop(columns=columns_to_drop, inplace=True)

In [27]:
train.shape

(120000, 31)

In [30]:
from sklearn.impute import SimpleImputer

# Identify numeric columns with missing values
numeric_columns_with_missing = train.select_dtypes(include=['number']).columns[train.select_dtypes(include=['number']).isnull().any()]

# Create a SimpleImputer with a strategy (e.g., 'mean', 'median', or 'most_frequent')
imputer = SimpleImputer(strategy='median')  # You can choose a different strategy if needed

# Impute missing values in numeric columns only
train[numeric_columns_with_missing] = imputer.fit_transform(train[numeric_columns_with_missing])



In [31]:
numeric_columns_with_missing

Index(['totals.pageviews'], dtype='object')

In [35]:
missing_pageviews = train['totals.pageviews'].isnull().sum()
missing_pageviews

0